## Gestión inicial de vectores en Pinecone

### Inicialización de variables

In [1]:
# Carga de variables de entorno
import os
from dotenv import load_dotenv
load_dotenv()

True

### Inicialización de embeddings

In [2]:
### EMBEDDINGS
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
embedding_size = embedding_model._client.get_sentence_embedding_dimension()
print(f"Tamaño teórico del embedding: {embedding_size}. Tamaño real del embedding: {len(embedding_model.embed_query('hola'))}.")

Tamaño teórico del embedding: 768. Tamaño real del embedding: 768.


### Carga de documentos que estarán disponibles para el chatbot en la base de datos vectorial Pinecone

In [3]:
# Cargar documentos PDF
from langchain_community.document_loaders import PyPDFDirectoryLoader

def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

docs_dir = os.getenv("DOCS_DIR") or "./docs"
docs = read_doc(docs_dir)

print(f"Páginas cargadas: {len(docs)}")

Páginas cargadas: 3


### Fragmentación de documentos

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_data(docs, chunk_size=800, chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

chunks=chunk_data(docs=docs,chunk_size=1000, chunk_overlap=50)
print(f"Cantidad de fragmentos: {len(chunks)}")

Cantidad de fragmentos: 5


### Carga inicial de documentos a Pinecone

In [5]:
## CONNECT WITH PINECONE DATABASE
from pinecone import Pinecone, ServerlessSpec
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY not found in environment variables") 

#Connect to Pinecone DB
pc=Pinecone(api_key=PINECONE_API_KEY)
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)
index_name = os.environ.get('PINECONE_INDEX_NAME') or 'ceia-2025-b5-pnl2-tp2'
recreate_index = os.environ.get('PINECONE_RECREATE_INDEX') or False

if recreate_index and index_name in pc.list_indexes().names():
  pc.delete_index(index_name)
  print(f"Index {index_name} borrado.")

# check if index already exists
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=embedding_size, # dimensionality of embeddings
        metric='cosine',
        spec=spec
        )
    print(f"Index creado con el nombre: {index_name}.")
else:
    print(f"El index con el nombre {index_name} ya estaba creado.")

Index ceia-2025-b5-pnl2-tp2 borrado.
Index creado con el nombre: ceia-2025-b5-pnl2-tp2.


In [6]:
## UPSERT THE VECTORS IN TO THE PINECONE DATABASE

from langchain_pinecone import PineconeVectorStore
namespace = "documentos"

docsearch = PineconeVectorStore.from_documents(
    documents=chunks,
    index_name=index_name,
    embedding=embedding_model, 
    namespace=namespace
)
print(f"Upserted values to {index_name} index")

Upserted values to ceia-2025-b5-pnl2-tp2 index


### Prueba de carga inicial con consultas a la base de datos de Pinecone

In [7]:
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embedding_model,
    namespace=namespace,
)
retriever=vectorstore.as_retriever()


In [8]:
query = "in which companies did Rob Otto used to work"
vectorstore.similarity_search(query, k=1)

[Document(id='649fbc02-671a-45dc-ac2b-3561b2bfd00d', metadata={'author': 'Gustavo Viñas', 'creationdate': '2025-11-25T22:05:05-03:00', 'creator': 'Microsoft® Word for Microsoft 365', 'moddate': '2025-11-25T22:05:05-03:00', 'page': 0.0, 'page_label': '1', 'producer': 'Microsoft® Word for Microsoft 365', 'source': 'docs\\cvRobOtto.pdf', 'total_pages': 3.0}, page_content='Rob Otto \nIngeniero de Machine Learning & Especialista en IA     Buenos Aires, Argentina \n(Disponibilidad Remota/Híbrida)         rob.otto@email.ficticio |    \nlinkedin.com/in/rob.otto.no.me.busquen |            github.com/rob.otto.noexiste.perfil  \nPerfil Profesional \nIngeniero de Software especializada en Inteligencia Artificial con más de 4 años de \nexperiencia en el ciclo de vida completo de modelos de Machine Learning. Experta en el \ndesarrollo de soluciones de Procesamiento de Lenguaje Natural (NLP) y Visión por \nComputadora, con un enfoque reciente en la implementación de LLMs (Large Language \nModels) en 

### Prueba básica de chatbot

In [9]:
from langchain_groq import ChatGroq

GROQ_API_KEY=os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY not found in environment variables") 

chat = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=GROQ_API_KEY)

In [10]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# 1. Crear la cadena que procesa los documentos ("stuff")
prompt = ChatPromptTemplate.from_template("""Answer based in context:\n\n{context}\n\Question: {input}""")
combine_docs_chain = create_stuff_documents_chain(chat, prompt)

# 2. Crear la cadena de recuperación completa
qa_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 3. Ejecutar
result = qa_chain.invoke({"input": query})
print(result["answer"])

According to the provided information, Rob Otto has worked in the following companies:

1. TechNova Solutions (Fintech) - Senior AI Engineer (Noviembre 2022 – Presente)
2. Retail Analytics Corp - Data Scientist (Junio 2020 – Octubre 2022)


In [11]:
query = "does Rob Otto have any hands-on experience"

result = qa_chain.invoke({"input": query})
print(result["answer"])


Based on the provided information, Rob Otto has hands-on experience with the following:

1. **Keras**: He is listed as one of the tools he is familiar with under the "Habilidades Técnicas" (Technical Skills) section.
2. **XGBoost**: Also listed as one of the tools he is familiar with under the "Habilidades Técnicas" section.
3. **Fine-tuning de LLMs (Llama 3, Mistral)**: He has experience fine-tuning LLMs, specifically mentioning Llama 3 and Mistral, in his project "LegalSumm AI" and in his role as a Senior AI Engineer at TechNova Solutions.
4. **RAG (Retrieval-Augmented Generation)**: He designed and implemented a RAG system using LangChain and OpenAI API in his role as a Senior AI Engineer at TechNova Solutions.
5. **LangChain**: He used LangChain in his project "LegalSumm AI" and in his role as a Senior AI Engineer at TechNova Solutions.
6. **Prompt Engineering**: Although not explicitly mentioned, his experience with fine-tuning LLMs and designing RAG systems implies familiarity wi

In [12]:
print(result)

{'input': 'does Rob Otto have any hands-on experience', 'context': [Document(id='3b94ec57-219c-4338-908d-df473581aec5', metadata={'author': 'Gustavo Viñas', 'creationdate': '2025-11-25T22:05:05-03:00', 'creator': 'Microsoft® Word for Microsoft 365', 'moddate': '2025-11-25T22:05:05-03:00', 'page': 0.0, 'page_label': '1', 'producer': 'Microsoft® Word for Microsoft 365', 'source': 'docs\\cvRobOtto.pdf', 'total_pages': 3.0}, page_content='Keras, XGBoost. \n• IA Generativa: Fine-tuning de LLMs (Llama 3, Mistral), RAG (Retrieval-Augmented \nGeneration), LangChain, Prompt Engineering. \n• MLOps & Cloud: AWS (SageMaker, Lambda), Azure ML, Docker, Kubernetes, MLflow, \nDVC (Data Version Control). \n• Bases de Datos: PostgreSQL, MongoDB, Pinecone (Vector DB), ChromaDB. \n• Matemáticas: Álgebra Lineal, Cálculo Multivariable, Estadística Bayesiana. \n \nExperiencia Laboral \nSenior AI Engineer | TechNova Solutions (Fintech) \nNoviembre 2022 – Presente \nLidero la iniciativa de automatización de at